In [2]:
import re 
from random import randint

**Classifier_Representation_Parameters_type**  

Classifier:
    - SVM
    - Other  
Representation:
    - L = lemma  
    - P = pos Tag    
    - R = rs and disease tag
Parameters position:
    1. Vectorizer
        (1) TFIDF 
        (2) BINARY 
        (3) TFIDFBINARY
    3. ngrams:
        (1) 1
        (2) 2
        (3) 3
        (4) 1,2
        (5) 1,3
    4.Class weight
        (0) unbalanced
        (1) balanced
    5. Remove stop words
        (0) False
        (1) True
        
SVM:  

     1.Kernel  
        (1) linear  
        (2) poly  
        (3) rbf  
MLP:

    1.Hidden Layers
        (1) one 
        (2) one
    2.Activation Formula
        (1) Linear
        (2) tanh
        (3) relu

**Command lines description **  

***Basic***      

    --inputPath >>> mainInPath  
    --inputTrainingClasses >>>  
    --outputPath >>> mainOutPath  
    --positiveClass >>> DISEASE
    --sgram
    --fgram
    
***Combinatory***     

    --inputTrainingSentences >>> trainS  
    --outputFile >>> outf  
    --classifier >>> classifier(SVM,O)  
    --removeStopWords >>> remStopWds (TRUE,FALSE)   
    --vectype >>> vectorizer (TFIDF,BINARY,TFIDFBINARY)  
    --classweight >>> classweight (balanced,unbalanced) 
    
***SVM only***   

    --kernel >>> kernel  
    
***MLP only***

    --hiddenlayers >>> hlayers (1,2)
    --activation >>> activation (linear,tanh,relu)

#### Building input paths

In [27]:
mainInPath = r'/home/larisams/DataMin/Training/inputData'
mainOutPath = r'/home/larisams/DataMin/Training/outputData'
trainS = [r'training_AC_LRK.txt',r'training_AC_LR.txt']
inTrainClass = r'training-classes.txt'
params = ['--']

#### Building parameters dictionay 

In [28]:
vectorizer = {'TFIDF':'1','BINARY':'2','TFIDFBINARY':'3'}
remStopWds = {'T':'1','F':'0'} 
ngrams = {'1':['1'],'2':['2'],'3':['3'],'4':['1','2'],'5':['1','3']}
classweight = {'unbalanced':'0','balanced':'1'}
kernel = {'linear':'1','poly':'2','rbf':'3'}
script = 'Lmod_training-cross-validation-improving.py'
BasicParm = ' '.join(['python3',str(script),'--inputPath',str(mainInPath),'--inputTrainingClasses',str(inTrainClass),'--outputPath',str(mainOutPath),'--positiveClass','DISEASE'])
CP = ['--inputTrainingSentences','--outputFile','--classifier','--vectype','--classweight','--sngram','--fngram']
activationF = {'identity':'1','tanh':'2','relu':'3'}

### Grid MLP

In [29]:
cmline,outf ='',''
i=0
grid = open(r'/Users/larisams/Dropbox/Projects/BI/MineriaDatos/Training-CV/Grid_MLP.sh','w')
for ts in trainS:
    for vec in vectorizer:
        for ng in ngrams:
            for ac in activationF:
                for cw in classweight:
                    for sw in remStopWds:
                        outf,cmline,sgr,fgr = '','','',''
                        sgr= ngrams[ng][0]
                        fgr= ngrams[ng][0]
                        if len(ngrams[ng])>1:
                            fgr = ngrams[ng][1]
                        hl = str(randint(100,250))
                        outf = 'MLP'+'_'+re.search(r'_(.*).txt',ts).group(1)+'_'+vectorizer[vec]+ng+activationF[ac]+classweight[cw]+remStopWds[sw]+'_'+hl+'.txt'
                        cmline = ' '.join([BasicParm,CP[0],ts,CP[1],outf,CP[2],'MLP',CP[3],vec,CP[4],cw,CP[5],sgr,CP[6],fgr,"--hiddenlayer",hl+',','--activation',ac])
                        if remStopWds[sw] == '1':
                            cmline = ' '.join([cmline,"--removeStopWords"])
                        cat = 'cat {} {} > ed && mv ed {} | rm {}\n'.format('/'.join([mainOutPath,'FinalReport.txt']),'/'.join([mainOutPath,outf]),'/'.join([mainOutPath,'FinalReport.txt']),'/'.join([mainOutPath,outf]))
                        grid.write(cmline+'\n'+cat)
                        i+=1
grid.close()

### Grid SVM

In [30]:
cmline,outf ='',''
i=0
grid = open(r'/Users/larisams/Dropbox/Projects/BI/MineriaDatos/Training-CV/Grid_SVM.sh','w')
for ts in trainS:
    for vec in vectorizer:
        for ng in ngrams:
            for ker in kernel:
                for cw in classweight:
                        for sw in remStopWds:
                            outf,cmline = '',''
                            outf = 'SVM'+'_'+re.search(r'_(.*).txt',ts).group(1)+'_'+vectorizer[vec]+ng+classweight[cw]+kernel[ker]+remStopWds[sw]+'.txt'
                            cmline = ' '.join([BasicParm,CP[0],ts,CP[1],outf,CP[2],'SVM',CP[3],vec,CP[4],cw,CP[5],sgr,CP[6],fgr,'--kernel',ker])
                            if remStopWds[sw] == '1':
                                cmline = ' '.join([cmline,"--removeStopWords"])
                            cat = 'cat {} {} > ed && mv ed {} | rm {}\n'.format('/'.join([mainOutPath,'FinalReport.txt']),'/'.join([mainOutPath,outf]),'/'.join([mainOutPath,'FinalReport.txt']),'/'.join([mainOutPath,outf]))
                            grid.write(cmline+'\n'+cat)
                            i+=1
grid.close()